https://towardsdatascience.com/creating-high-resolution-satellite-images-with-mapbox-and-python-750b3ac83dd7

https://www.kaggle.com/kapastor/high-resolution-mapping-with-mapbox-and-three-js

In [ ]:
# Step 1: Install of the packages
!pip install mercantile

In [ ]:
# The first step is to define the min/max (lat,lng)
# Go to google maps and select Whats here to see the top left and bottom right sets.
# Set the zoom level (resolution level)
lat_lng = [43.640918, -79.371478]
delta=0.05
tl = [lat_lng[0]+delta, lat_lng[1]-delta]
br = [lat_lng[0]-delta, lat_lng[1]+delta]
z = 14 # Set the resolution (max at 15)

In [ ]:
# find the tile set IDs (x/y) for the top left and bottom right at the zoom level
import mercantile
tl_tiles = mercantile.tile(tl[1],tl[0],z)
br_tiles = mercantile.tile(br[1],br[0],z)
x_tile_range =[tl_tiles.x,br_tiles.x];print(x_tile_range)
y_tile_range = [tl_tiles.y,br_tiles.y];print(y_tile_range)

In [ ]:
token = 'pk.eyJ1Ijoibmljb2pnIiwiYSI6ImNrcDc1bGhtdDA3cDAyd21qZnR2MTE4dTcifQ.P02LX2SUoF-EirCxFWZaqQ'

In [ ]:
# Make the folders
!mkdir ./satellite_images
!rm -rf ./satellite_images/*
!mkdir ./elevation_images
!rm -rf ./elevation_images/*

In [ ]:
import requests # The requests package allows use to call URLS
import shutil   # shutil will be used to copy the image to the local

# Loop over the tile ranges
for i,x in enumerate(range(x_tile_range[0],x_tile_range[1]+1)):
    for j,y in enumerate(range(y_tile_range[0],y_tile_range[1]+1)):
        
        # Call the URL to get the image back
        r = requests.get('https://api.mapbox.com/v4/mapbox.terrain-rgb/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token='+token, stream=True)
        
        # Next we will write the raw content to an image
        with open('./elevation_images/' + str(i) + '.' + str(j) + '.png','wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 
            
        # Do the same for the satellite data
        r =requests.get('https://api.mapbox.com/v4/mapbox.satellite/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token='+token, stream=True)
        with open('./satellite_images/' + str(i) + '.' + str(j) + '.png','wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

In [ ]:
# Combine the tiles into a single large image
!mkdir ./composite_images
!mkdir ./animate

In [ ]:
from PIL import Image
import math
from os import listdir
from os.path import isfile, join

for img_name in ['elevation','satellite']:
    image_files = ['./'+img_name+'_images/' + f for f in listdir('./'+img_name+'_images/')]
    images = [Image.open(x) for x in image_files]

    edge_length_x = x_tile_range[1] - x_tile_range[0]
    edge_length_y = y_tile_range[1] - y_tile_range[0]
    edge_length_x = max(1,edge_length_x)
    edge_length_y = max(1,edge_length_y)
    width, height = images[0].size

    total_width = width*edge_length_x
    total_height = height*edge_length_y

    composite = Image.new('RGB', (total_width, total_height))
    print(total_width,total_height)

    anim_idx = 0
    y_offset = 0
    for i in range(0,edge_length_x):
        x_offset = 0
        for j in range(0,edge_length_y):
            tmp_img = Image.open('./'+img_name+'_images/' + str(i) + '.' + str(j) + '.png')
            composite.paste(tmp_img, (y_offset,x_offset))
            x_offset += width
            composite.save('./animate/'+str(anim_idx).zfill(4)+'.jpg',optimize=True,quality=95)
            anim_idx += 1
            print(anim_idx)

            
        y_offset += height

    composite.save('./composite_images/'+img_name+'.png')

In [ ]:
!apt-get update
!apt install -y ffmpeg
!rm output.mp4
!ffmpeg  -i ./animate/%04d.jpg -c:v libx264 -c:a aac -ar 44100  -pix_fmt yuv420p output.mp4
        
# ffmpeg -ss 30 -t 3 -i input.mp4 -vf "fps=10,scale=320:-1:flags=lanczos,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse" -loop 0 output.gif

In [ ]:
from PIL import Image
elevation_raw = Image.open('./composite_images/elevation.png')
rgb_elevation = elevation_raw.convert('RGBA')

# Loop over the image and save the data in a list:
elevation_data = []
# texture_data = []
for h in range(rgb_elevation.height):
    for w in range(rgb_elevation.width):
        R, G, B, A = rgb_elevation.getpixel((w, h))
        height = -10000 + ((R * 256 * 256 + G * 256 + B) * 0.1)
        elevation_data.append(height)

import json
with open('./elevation.json', 'w') as outfile:
    json.dump(elevation_data, outfile)

# Test Static Images API
https://docs.mapbox.com/playground/static/

In [ ]:
import requests

In [ ]:
token = 'pk.eyJ1Ijoibmljb2pnIiwiYSI6ImNrcDc1bGhtdDA3cDAyd21qZnR2MTE4dTcifQ.P02LX2SUoF-EirCxFWZaqQ'

In [ ]:
lat = -118.1227
lng = 33.83
zoom = 20
width = 1280
height = 1280

In [ ]:
url = f'https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/{lat},{lng},{zoom},0/{width}x{height}?access_token={token}'
r = requests.get(url, stream=True)

In [ ]:
!mkdir ./test_static

In [ ]:
import shutil
with open('./test_static/test.png','wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 

In [ ]:
from IPython.display import Image
Image(filename='./test_static/test.png') 